# Decision analysis with `DecisionProgramming.jl` using Petitti, 2000, 2e. - Chapter 2
Tomás Aragón, Updated 2026-01-20

Chapter 2 from Petitti, Diana B. Meta-Analysis, Decision Analysis, and Cost-Effectiveness Analysis: Methods for Quantitative Synthesis in Medicine. 2nd ed. Monographs in Epidemiology and Biostatistics, v. 31. Oxford University Press, 2000. https://doi.org/10.1093/acprof:oso/9780195133646.001.0001.

This Jupyter Julia notebook was created in VS Code and contains examples from Petitti 2000, Chapter 2 using Julia code. You can run this notebook in VS Code, Positron, or JupyterLab with a Julia kernel. 

We will use `DecisionProgramming.jl` Julia package: https://gamma-opt.github.io/DecisionProgramming.jl/dev/

Preparation:
- Read [Decision analysis with `rdecision` using Petitti, 2000, 2e. - Chapter 2]() 
- Read [Section 2.2 of Chapter 2](https://github.com/tomasaragon/di4h/blob/main/files/Decision-Analysis_Petitti_2000_isbn_9780195133646_ch02.pdf) in Petitti 2000 book. 
- Review [Introduction to the construction of decision trees](https://cran.r-project.org/web/packages/rdecision/vignettes/DT00-DecisionTreeTutorial.html) from the `rdecision` package CRAN page.  

Optional readings for background:
- Owens, Douglas K. “Analytic Tools for Public Health Decision Making.” Medical Decision Making: An International Journal of the Society for Medical Decision Making 22, no. 5 Suppl (2002): S3-10. https://doi.org/10.1177/027298902237969.
- Owens, D. K., R. D. Shachter, and R. F. Nease. “Representation and Analysis of Medical Decision Problems with Influence Diagrams.” Medical Decision Making: An International Journal of the Society for Medical Decision Making 17, no. 3 (1997): 241–62. https://doi.org/10.1177/0272989X9701700301.
- Nease, R. F., and D. K. Owens. “Use of Influence Diagrams to Structure Medical Decisions.” Medical Decision Making: An International Journal of the Society for Medical Decision Making 17, no. 3 (1997): 263–75. https://doi.org/10.1177/0272989X9701700302.
- Neapolitan, Richard, Xia Jiang, Daniela P. Ladner, and Bruce Kaplan. “A Primer on Bayesian Decision Analysis With an Application to a Kidney Transplant Decision.” Transplantation 100, no. 3 (2016): 489–96. https://doi.org/10.1097/TP.0000000000001145.

https://gamma-opt.github.io/DecisionProgramming.jl/dev/examples/used-car-buyer/ 

<figure>
<img src="Petitti_2000_2e_ch02_fig2-6.png" width="800" alt="Petitti 2000 Figure 2-6"/>
<figcaption>FIGURE 1</figcaption>
</figure>

<figure>
<img src="img_measles_decision_network_drawio.png" width="500" alt="Petitti 2000 Figure 2-1"/>
<figcaption>FIGURE 2</figcaption>
</figure>

In [1]:
using JuMP, HiGHS
using DecisionProgramming
diagram = InfluenceDiagram(); 


## Set nodes, incoming edges, and states

In [2]:
add_node!(diagram, DecisionNode("R", [], ["R1", "R0"])) # "R1" = revax, "R0" = no revax
add_node!(diagram, ChanceNode("E", [], ["E1", "E0"])) # "E1" = exposed, "E0" = not exposed
add_node!(diagram, ChanceNode("M", ["R", "E"], ["M1", "M0"])) # "M1" = measles, "M0" = no measles
add_node!(diagram, ChanceNode("D", ["M"], ["D1", "D0"])) # "D1" = dead, "D0" = well 
add_node!(diagram, ValueNode("V", ["R", "D"]));

## Assign probabilities to variables 

In [ ]:
# probabilities
# Pr(exposure to measles)
p_E1 = 0.20
p_E0 = 1 - p_E1

# Pr(measles | exposure, revax)
# revax arm
p_M1_E1_R1 = 0.05 # 
p_M0_E1_R1 = 1 - p_M1_E1_R1
p_M1_E0_R1 = 0.00
p_M0_E0_R1 = 1 - p_M1_E0_R1

# no revax arm
p_M1_E1_R0 = 0.33
p_M0_E1_R0 = 1 - p_M1_E1_R0
p_M1_E0_R0 = 0.00
p_M0_E0_R0 = 1 - p_M1_E0_R0

# Pr(death | measles)
p_D1_M1 = 0.0023
p_D0_M1 = 1 - p_D1_M1
p_D1_M0 = 0.00 
p_D0_M0 = 1 - p_D1_M0

1.0

## Generate arcs 

In [4]:
generate_arcs!(diagram) 

OrderedCollections.OrderedDict{String, Utilities}()

### options for viewing diagram attributes  
- diagram.Nodes 
- diagram.Names 
- diagram.I_j 
- diagram.States 
- diagram.S 
- diagram.C 
- diagram.D 
- diagram.V 

In [5]:
diagram.I_j 

OrderedCollections.OrderedDict{String, Vector{String}} with 5 entries:
  "R" => []
  "E" => []
  "M" => ["R", "E"]
  "D" => ["M"]
  "V" => ["R", "D"]

In [ ]:
# add_node!(diagram, DecisionNode("R", [], ["y", "n"]))

# add_node!(diagram, ChanceNode("E", [], ["E1", "E0"]))
X_E = ProbabilityMatrix(diagram, "E")
X_E["E1"] = 0.2
X_E["E0"] = 0.8
add_probabilities!(diagram, "E", X_E)

# add_node!(diagram, ChanceNode("M", ["E", "R"], ["M1", "M0"]))
X_M = ProbabilityMatrix(diagram, "M")
X_M["R1","E1","M1"] = p_M1_E1_R1
X_M["R1","E1","M0"] = 1 - p_M1_E1_R1
X_M["R1","E0","M1"] = p_M1_E0_R1
X_M["R1","E0","M0"] = 1 - p_M1_E0_R1
X_M["R0","E1","M1"] = p_M1_E1_R0
X_M["R0","E1","M0"] = 1 - p_M1_E1_R0
X_M["R0","E0","M1"] = p_M1_E0_R0
X_M["R0","E0","M0"] = 1 - p_M1_E0_R0
add_probabilities!(diagram, "M", X_M)

# add_node!(diagram, ChanceNode("D", ["M"], ["D1", "D0"]))
X_D = ProbabilityMatrix(diagram, "D")
X_D["M1","D1"] = p_D1_M1
X_D["M1","D0"] = 1 - p_D1_M1
X_D["M0","D1"] = p_D1_M0
X_D["M0","D0"] = 1 - p_D1_M0
add_probabilities!(diagram, "D", X_D)

# add_node!(diagram, ValueNode("V", ["R", "D"]))
Y_V = UtilityMatrix(diagram, "V")
Y_V["R1","D1"] = 0
Y_V["R1","D0"] = 1
Y_V["R0","D1"] = 0
Y_V["R0","D0"] = 1 
add_utilities!(diagram, "V", Y_V)

2×2 Utilities{2}:
 0.0  1.0
 0.0  1.0

In [7]:
model, z, variables = generate_model(diagram, model_type="RJT")

(A JuMP Model
├ solver: none
├ objective_sense: MAX_SENSE
│ └ objective_function_type: AffExpr
├ num_variables: 35
├ num_constraints: 83
│ ├ AffExpr in MOI.EqualTo{Float64}: 46
│ ├ AffExpr in MOI.LessThan{Float64}: 2
│ ├ VariableRef in MOI.GreaterThan{Float64}: 33
│ └ VariableRef in MOI.ZeroOne: 2
└ Names registered in the model
  └ :EV, OrderedCollections.OrderedDict{String, DecisionProgramming.DecisionVariable}("R" => DecisionProgramming.DecisionVariable("R", String[], VariableRef[R_1, R_2])), RJTVariables(Dict{String, DecisionProgramming.μVariable}("M" => DecisionProgramming.μVariable("M", VariableRef[M[R1, E1, M1] M[R1, E0, M1]; M[R0, E1, M1] M[R0, E0, M1];;; M[R1, E1, M0] M[R1, E0, M0]; M[R0, E1, M0] M[R0, E0, M0]]), "D" => DecisionProgramming.μVariable("D", VariableRef[D[R1, M1, D1] D[R1, M0, D1]; D[R0, M1, D1] D[R0, M0, D1];;; D[R1, M1, D0] D[R1, M0, D0]; D[R0, M1, D0] D[R0, M0, D0]]), "E" => DecisionProgramming.μVariable("E", VariableRef[E[R1, E1] E[R1, E0]; E[R0, E1] E[R0, E0]

In [8]:
optimizer = optimizer_with_attributes(
    () -> HiGHS.Optimizer()
)
set_optimizer(model, optimizer)
optimize!(model)

Running HiGHS 1.12.0 (git hash: 755a8e027): Copyright (c) 2025 HiGHS under MIT licence terms
MIP has 48 rows; 35 cols; 131 nonzeros; 2 integer variables (2 binary)
Coefficient ranges:
  Matrix  [2e-03, 1e+00]
  Cost    [1e+00, 1e+00]
  Bound   [1e+00, 1e+00]
  RHS     [1e+00, 1e+00]
Presolving model
15 rows, 12 cols, 36 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve reductions: rows 0(-48); columns 0(-35); nonzeros 0(-131) - Reduced to empty
Presolve: Optimal

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic;
     I => Shifting; J => Feasibility jump; L => Sub-MIP; P => Empty MIP; R => Randomized rounding;
     S => Solve LP; T => Evaluate node; U => Unbounded; X => User solution; Y => HiGHS solution;
     Z => ZI Round; l => Trivial lower; p => Trivial point; u => Trivial upper; z => Trivial zero

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves 

In [9]:
Z = DecisionStrategy(diagram, z)
S_probabilities = StateProbabilities(diagram, Z)
U_distribution = UtilityDistribution(diagram, Z)

UtilityDistribution([0.0, 1.0], [2.3000000000000003e-5, 0.9999770000000001])

In [10]:
U_distribution

UtilityDistribution([0.0, 1.0], [2.3000000000000003e-5, 0.9999770000000001])

In [11]:
print_decision_strategy(diagram, Z, S_probabilities)

┌───────────────┐
│ Decision in R │
├───────────────┤
│ R1            │
└───────────────┘


In [12]:
print_utility_distribution(U_distribution)


┌──────────┬─────────────┐
│  Utility │ Probability │
│  Float64 │     Float64 │
├──────────┼─────────────┤
│ 0.000000 │    0.000023 │
│ 1.000000 │    0.999977 │
└──────────┴─────────────┘


In [13]:
print_statistics(U_distribution)

┌──────────┬──────────────┐
│     Name │   Statistics │
│   String │      Float64 │
├──────────┼──────────────┤
│     Mean │     0.999977 │
│      Std │     0.004796 │
│ Skewness │  -208.507220 │
│ Kurtosis │ 43473.260893 │
└──────────┴──────────────┘
